# Functions for Finding Pairs
Jiaxin (Jackie) Zhang
Find cointegrated stock pairs in Russell 3000, by sector.

  1. Generate ticker_list by sector (using Russell 3000 stock list). 
  2. Get Yahoo Finance daily data. Loop through Yahoo Finance daily data to find cointegration in longer term, e.g 3 months. Narrow down the pairs list.
  3. Read in pairs list generated in step 2, get IEX minute data, then check the cointegration in shorter time frame using IEX minute data. e.g 1 month of minute data. Then return the final dataframe for pairs found.

In this way, we are able to reduce the running time. Miniute data is very computative expensive. Also, I used two ways to check contegration. For daily data, I use ADF test. For miniute data, use coint() function in statsmodel, which utilize the augmented Engle-Granger two-step cointegration test. Only pairs that passes both test can be added to our list.

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from pandas_datareader import DataReader
import pprint
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from statsmodels.api import OLS
import statsmodels.tsa.stattools as ts
from datetime import datetime, timedelta
from iexfinance.stocks import Stock, get_historical_intraday, get_historical_data


#IEX_TOKEN = (write your IEX token here)

## Functions

### Read Data

In [31]:
def get_ticker_list(sector):
    df = pd.read_excel('Russell_3000_stock_list.xlsx',skiprows = range(0,7))
    #need ticker, sector, asset class, market value info
    Russell_3000 = df[['Ticker','Name','Sector','Asset Class','Price']]
    Russell_3000 = Russell_3000[Russell_3000['Asset Class'] == 'Equity']

    #Get stocks of specific sector 
    sector_data = Russell_3000[Russell_3000['Sector'] == sector ]
    ticker_list = sector_data.loc[:,'Ticker']
    return ticker_list

def get_daily_data(ticker_list,start,end):
    data_all = pd.DataFrame()
    for ticker in ticker_list:
        try:
            print("Get data: ",ticker)
            data = DataReader(ticker, "yahoo", start, end)
            data_all[ticker] = data["Adj Close"]
        except:
            print("Can't find ",ticker)
            break
    data_all.to_csv("Daily_data.csv")
    return data_all


def get_miniute_data(ticker_list,end,period,IEX_TOKEN = "sk_00bc5a026ca647c78c97f428efc823b7"):    
    dates = [end - timedelta(n) for n in range(period, 1, -1)]
    data_all = pd.DataFrame()
    
    for ticker in ticker_list:
        stock = pd.DataFrame()
        print(ticker)
        for date in dates:
            try:
                data = get_historical_intraday(ticker, date, output_format="pandas", token=IEX_TOKEN)
                stock = pd.concat([stock, data])
            except:
                print("Can't find ",ticker)
                break
        if len(stock) > 0:
            data_all[ticker] = stock["marketAverage"]
    data_all.to_csv("Miniute_data.csv")
    return data_all

### CADF_test for daily data

In [36]:
def CADF_test(df1,df2):
    '''
    read in two lists of prices.
    return p-value
    '''
    # Calculate optimal hedge ratio "beta"
    res = OLS(df1,df2).fit()
    beta_hr = res.params[0]

    # Calculate the residuals of the linear combination
    res = df2 - beta_hr*df1

    # Calculate and output the CADF test on the residuals
    cadf = ts.adfuller(res)
    #return p-value
    return cadf[1]

def find_cointegrated_pairs_daily(data):
    pairs = []
   #deal with missing data
    data = data.dropna(thresh=len(data)-50,axis=1) #only keep the columns with less than 50 missin values
    data = data.fillna(method = 'ffill')
    n = data.shape[1]
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            #test coint here
            pvalue = CADF_test(S1, S2)
            #pvalue = ts.coint(S1, S2)[1]
            print('test '+keys[i]+' '+keys[j]+" P-value "+str(pvalue))
            #set threshold here
            if pvalue < 0.02:
                pairs.append([keys[i], keys[j],pvalue])
    pairs_df = pd.DataFrame(pairs, columns = ['Ticker1', 'Ticker2','P-value'])
    pairs_df.to_csv("Pairs_daily.csv")
    return pairs_df

### Test for Minute data

In [25]:
def find_cointegrated_pairs_min(data):
    '''
    data: the data frame that contains all the stock price data you want to tested out. Each column is for one ticker.
    It's faster then simply read in data for each iteration
    '''
    #how to deal with missing data?
    data = data.dropna(thresh=len(data)-200,axis=1) #only keep the columns with less than 200 missin values
    data = data.fillna(method = 'ffill')
    n = data.shape[1]
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            #test coint here
            pvalue = ts.coint(S1, S2)[1]
            print('test '+keys[i]+' '+keys[j]+" P-value "+str(pvalue))
            #set threshold here
            if pvalue < 0.02:
                pairs.append([keys[i], keys[j],pvalue])
                
    pairs_df = pd.DataFrame(pairs, columns = ['Ticker1', 'Ticker2','P-value'])
    pairs_df.to_csv("Pairs_minute.csv")
    return pairs_df

## Test on daily data from yahoo finance

In [21]:
#create a test ticker list
start = datetime(2020, 4, 1)
end = datetime(2020, 7, 1)
ticker_list = ['JNJ', 'BSX','UNH', 'MYL']
daily_data = get_daily_data(ticker_list,start,end)
daily_data

Get data:  JNJ
Get data:  BSX
Get data:  UNH
Get data:  MYL


,JNJ,BSX,UNH,MYL
Date,,,,
2020-04-01,127.923225,30.190001,236.302277,14.260000
2020-04-02,132.233337,31.049999,239.408890,13.920000
2020-04-03,133.246323,31.170000,228.505859,13.740000
2020-04-06,138.797836,32.880001,247.275009,14.040000
2020-04-07,136.533539,33.689999,246.976288,14.370000
...,...,...,...,...
2020-06-25,139.669998,33.889999,296.220001,16.379999
2020-06-26,137.809998,33.279999,286.880005,15.890000
2020-06-29,139.039993,34.650002,289.760010,15.990000


In [24]:
pairs_df = find_cointegrated_pairs_daily(daily_data)
pairs_df

test JNJ BSX P-value 0.008654464254582536
test JNJ UNH P-value 0.07309767072080597
test JNJ MYL P-value 0.00635551043440394
test BSX UNH P-value 0.03089408513568859
test BSX MYL P-value 0.00036031067143563005
test UNH MYL P-value 0.02931078812168662


,Ticker1,Ticker2,P-value
0,JNJ,BSX,0.008654
1,JNJ,MYL,0.006356
2,BSX,MYL,0.000360


## Test on Minute data

In [35]:
end = datetime(2020, 7, 1)
ticker_list = ['JNJ', 'BSX','UNH', 'MYL']
min_data = get_miniute_data(ticker_list,end,30)  #meaning 6/1-7/1 (30days)
min_data

JNJ
BSX
UNH
MYL


,JNJ,BSX,UNH,MYL
2020-06-01 09:30:00,147.359,37.742,303.805,17.049
2020-06-01 09:31:00,147.781,37.873,303.079,17.235
2020-06-01 09:32:00,147.935,38.014,303.263,17.290
2020-06-01 09:33:00,148.053,38.038,302.943,17.288
2020-06-01 09:34:00,148.005,37.979,302.849,17.380
...,...,...,...,...
2020-06-29 15:55:00,138.559,34.524,288.786,15.944
2020-06-29 15:56:00,138.584,34.532,288.688,15.946
2020-06-29 15:57:00,138.646,34.537,288.868,15.955
2020-06-29 15:58:00,138.724,34.540,288.848,15.954


In [38]:
pairs_df_min = find_cointegrated_pairs_min(min_data)
pairs_df_min 

test JNJ BSX P-value 0.009802306751239534
test JNJ UNH P-value 0.3643787340714213
test JNJ MYL P-value 0.11300679985946516
test BSX UNH P-value 0.693852730822063
test BSX MYL P-value 0.526487321094211
test UNH MYL P-value 0.014910145452891902


,Ticker1,Ticker2,P-value
0,JNJ,BSX,0.009802
1,UNH,MYL,0.014910
